In [24]:
from bs4 import BeautifulSoup
import requests
from collections import defaultdict
import pandas as pd
import time
import urllib2

pages = {}
url = "http://www.capterra.com/project-management-software/"
page = urllib2.urlopen(url)
soup = BeautifulSoup(page.read())
r = requests.get(url)
pages[url] = r.text


In [25]:
products = soup.findAll('div', {'class': 'card  zindex-hack  listing'})
print len(products)


455


In [28]:
urls = []
lis = soup.findAll('a',{'class':'spotlight-link'})
for li in lis: 
    urls.append('http://www.capterra.com' + li['href'])

In [43]:
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    pages[url] = soup
    time.sleep(0.5)

KeyboardInterrupt: 

In [45]:
len(pages)

423

In [73]:
rows = []
cols = []
for url in urls[0:421]:
    soup = pages[url]
    name = soup.find('h1', {'itemprop': 'name'}).text
    
    all_features = []
    has_feature = []
    missing_feature = []
    checklist = soup.findAll('ul', {'class': 'check-list'})
    try:
        close_to_price = checklist[1].findAll('div', {'class': 'cell seven-twelfths'})
        rawprice= close_to_price[0].text.replace('See pricing details', '').strip()
    except IndexError:
        rawprice = ''
    
    i = rawprice.find('/')
    try:
        price = float(rawprice[0:i].replace('$', ''))
        unit = rawprice[i:len(rawprice)]
    except ValueError:
        price = 0
        unit = rawprice
    
    lis = soup.findAll('li', {'class': 'ss-check'})
    for li in lis:
        has_feature.append(li.text)
    lis = soup.findAll('li', {'class': 'ss-delete color-gray'})
    for li in lis:
        missing_feature.append(li.text)
    has_feature_dict = defaultdict(int)
    for feature in has_feature:
        has_feature_dict[feature] = 1
    
    row = []
    row.append(name)
    if len(cols) == 0:
        ul = soup.find('ul', {'class': 'check-list  check-list-slim features-check-list'})
        cols = ['product_name']
        lis = ul.findAll('li')
        for li in lis:
            cols.append(li.text)
        cols.append('price')
        cols.append('units')
    for c in range(1, len(cols)):
        has = has_feature_dict[cols[c]]
        row.append(has)
    row.append(price)
    row.append(unit)
    rows.append(row)

In [78]:
print len(cols), df.shape

21 (421, 23)


In [79]:
print cols

['product_name', u'Budget Management', u'Bug Tracking', u'Collaboration', u'Email Integration', u'File Sharing', u'Gantt Charts', u'Idea Management', u'Issue Management', u'Milestone Tracking', u'Percent-Complete Tracking', u'Portfolio Management', u'Project Planning', u'Requirements Management', u'Resource Management', u'Status Tracking', u'Task Management', u'Testing / QA Management', u'Time & Expense Tracking', 'price', 'units']


In [81]:

df = pd.DataFrame(rows)
#df.columns = cols
df


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Salesforce,1,0,1,1,1,1,0,1,1,...,1,1,0,1,1,1,0,0,5.00,/month/user
1,Smartsheet,1,0,1,1,1,1,0,1,1,...,0,1,1,1,0,1,0,0,14.00,/month/user
2,Mavenlink,1,0,1,1,1,1,0,1,1,...,0,1,0,1,0,1,0,0,6.00,/month/user
3,Wrike,0,1,1,1,1,1,0,1,1,...,0,1,1,1,0,0,0,0,49.00,/month
4,WorkZone,1,0,1,1,1,1,0,0,1,...,0,1,1,1,0,1,0,0,200.00,/month
5,Easy Projects,1,1,1,0,1,1,0,1,1,...,1,1,1,1,1,1,0,0,15.75,/month/user
6,LiquidPlanner,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,29.00,/month/user
7,Procore Construction,1,0,1,1,1,1,0,1,1,...,1,1,1,1,1,1,0,0,0.00,"Yes\n , get a free trial"
8,Intervals,1,1,1,1,1,1,0,1,1,...,0,1,1,1,1,1,0,0,25.00,/month
9,Project Insight,1,0,1,1,1,1,0,1,1,...,0,1,0,1,0,1,0,0,0.00,"Yes\n , get a free demo"
